In [1]:
!nvidia-smi

Fri Aug 22 03:48:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


What is nvidia-smi?
It is the NVIDIA System Management Interface program.

Shows GPU status, driver versions, and other hardware info.

Installed with NVIDIA GPU drivers.*italicized text*

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge-score py7zr -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.6/413.6 kB 26.0 MB/s eta 0:00:00


Here is why each package is used:

transformers[sentencepiece]: Installs the Hugging Face Transformers library along with the optional SentencePiece tokenizer. Transformers provides pre-trained models (like BERT, GPT, PEGASUS) for a variety of NLP tasks. SentencePiece is a subword tokenizer commonly used for models that require tokenizing raw text in a way that supports multiple languages or unknown words.

datasets: A library by Hugging Face for easily loading, processing, and managing large datasets in a format optimized for NLP and machine learning workflows.

sacrebleu: A tool used to calculate BLEU scores, a standard metric to evaluate machine translation and text generation quality by comparing the generated text to reference texts.

rouge-score: A package to compute ROUGE metrics, which evaluate the quality of summaries by measuring overlap with reference summaries. This is key for summarization tasks.

py7zr: A library used to handle 7z compressed archive files, useful when datasets or models are distributed in 7z format.

-q: A pip option to run the installation quietly with minimal output.

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.55.2
Uninstalling transformers-4.55.2:
  Successfully uninstalled transformers-4.55.2
Found existing installation: accelerate 1.10.0
Uninstalling accelerate-1.10.0:
  Successfully uninstalled accelerate-1.10.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 28.5 MB/s eta 0:00:00


In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00


In [5]:
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import torch
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

1. from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
transformers is a popular library by Hugging Face for state-of-the-art pretrained models in NLP.

pipeline: A high-level API that simplifies running different NLP tasks (like text generation, translation, summarization) with just a few lines of code.

set_seed: A utility to set a random seed for reproducibility of results when running models with randomness.

AutoModelForSeq2SeqLM: A class that automatically loads a sequence-to-sequence language model (e.g., for translation or summarization).

AutoTokenizer: Automatically loads the proper tokenizer corresponding to the model, which converts text to tokens that the model can process.

2. from datasets import load_dataset, load_from_disk, load_metric
datasets is Hugging Face’s library for accessing and managing large datasets optimized for NLP/ML.

load_dataset: Loads a dataset from the Hugging Face Hub or local files.

load_from_disk: Loads a previously saved dataset from your local disk.

load_metric: Loads evaluation metrics (like BLEU, ROUGE) used to measure model output quality.

3. from nltk.tokenize import sent_tokenize
From the nltk (Natural Language Toolkit) library, sent_tokenize splits a given text into sentences. Useful for sentence-level processing.

4. import matplotlib.pyplot as plt
matplotlib.pyplot is a plotting library used to create visualizations like graphs and charts (commonly for displaying evaluation results).

5. import pandas as pd
pandas is a data manipulation library widely used to handle tabular data (dataframes). Useful for organizing and analyzing results.

6. from tqdm import tqdm
tqdm is a library that adds progress bars to loops, helpful for monitoring long-running operations like training or dataset processing.

7. import torch
torch is the core PyTorch machine learning framework used for building and running deep learning models (like the transformers you import above).

8. import nltk and nltk.download("punkt")
Imports the nltk toolkit.

Downloads the punkt tokenizer models needed for sentence splitting (sent_tokenize uses this data). This is required only once or on fresh setups.

Summary
This collection of imports and setup commands gives you:

Access to powerful pretrained NLP models and easy pipelines (transformers).

Tools to load, save, and evaluate datasets (datasets).

Sentence-level text processing (nltk).

Data manipulation and visualization (pandas, matplotlib).

Efficient progress display (tqdm).

Core deep learning framework (torch) for running and training models.

In [6]:
!!pip install evaluate


['Requirement already satisfied: evaluate in /usr/local/lib/python3.12/dist-packages (0.4.5)',
 'Requirement already satisfied: datasets>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from evaluate) (4.0.0)',
 'Requirement already satisfied: numpy>=1.17 in /usr/local/lib/python3.12/dist-packages (from evaluate) (2.0.2)',
 'Requirement already satisfied: dill in /usr/local/lib/python3.12/dist-packages (from evaluate) (0.3.8)',
 'Requirement already satisfied: pandas in /usr/local/lib/python3.12/dist-packages (from evaluate) (2.2.2)',
 'Requirement already satisfied: requests>=2.19.0 in /usr/local/lib/python3.12/dist-packages (from evaluate) (2.32.4)',
 'Requirement already satisfied: tqdm>=4.62.1 in /usr/local/lib/python3.12/dist-packages (from evaluate) (4.67.1)',
 'Requirement already satisfied: xxhash in /usr/local/lib/python3.12/dist-packages (from evaluate) (3.5.0)',
 'Requirement already satisfied: multiprocess in /usr/local/lib/python3.12/dist-packages (from evaluate) (0.70.1

In [7]:
import evaluate
metric = evaluate.load('rouge')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
import torch

In summary:
CUDA gives you the ability to dramatically speed up compute-intensive programs by using the GPU for general processing, not just for making graphics. This is crucial for modern AI, scientific computing, and any field that needs to process huge amounts of data quickly.

In [9]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [10]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

MODEL SELECTION

In [11]:
model_ckpt="google/pegasus-cnn_dailymail"
tokenizer=AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [12]:
print("Tokenizer size (vocab size):", tokenizer.vocab_size)

Tokenizer size (vocab size): 96103


In [13]:
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [14]:
dataset_samsum = load_dataset("knkarthick/samsum")

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [15]:
dataset_samsum["train"]["dialogue"][1]


'Olivia: Who are you voting for in this election? \nOliver: Liberals as always.\nOlivia: Me too!!\nOliver: Great'

In [16]:
dataset_samsum["train"][1]["summary"]

'Olivia and Olivier are voting for liberals in this election. '

✅ In short:

"train" → training split

[1] → second example in training data

["summary"] → only the summary text of that example

In [17]:
split_length=[len(dataset_samsum[split]) for split in dataset_samsum]
print(f"Split Length:{split_length}")
print(f"Features:{dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split Length:[14732, 818, 819]
Features:['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [18]:
'''def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }'''


"def convert_examples_to_features(example_batch):\n    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )\n\n    with tokenizer.as_target_tokenizer():\n        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )\n\n    return {\n        'input_ids' : input_encodings['input_ids'],\n        'attention_mask': input_encodings['attention_mask'],\n        'labels': target_encodings['input_ids']\n    }"

In [19]:
'''def convert_examples_to_features(example_batch):
    # Dialogue tokenize (input side)
    input_encodings = tokenizer(
        example_batch['dialogue'],
        max_length=1024,
        truncation=True,
        padding="max_length"   # padding add kiya for uniform length
    )

    # Summary tokenize (target side)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(
            example_batch['summary'],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
'''

'def convert_examples_to_features(example_batch):\n    # Dialogue tokenize (input side)\n    input_encodings = tokenizer(\n        example_batch[\'dialogue\'],\n        max_length=1024,\n        truncation=True,\n        padding="max_length"   # padding add kiya for uniform length\n    )\n\n    # Summary tokenize (target side)\n    with tokenizer.as_target_tokenizer():\n        target_encodings = tokenizer(\n            example_batch[\'summary\'],\n            max_length=128,\n            truncation=True,\n            padding="max_length"\n        )\n\n    return {\n        \'input_ids\': input_encodings[\'input_ids\'],\n        \'attention_mask\': input_encodings[\'attention_mask\'],\n        \'labels\': target_encodings[\'input_ids\']\n    }\n'

🔹 Why this works?

batched=True → tokenizer ek saath list of dialogues le lega aur unka batch bana dega.

padding="max_length" → har input ko fixed size banata hai (nahi toh sizes different ho jaayenge).

Ab .map() ek smooth processed dataset dega jisme har row ke paas tokens hote hain.

✅ In short:
Error isliye aaya kyunki tumhara function single string ke liye likha tha, lekin .map(batched=True) ne usse ek list pass ki.
Solution hai tokenizer ko list handle karne dena (jo wo already kar sakta hai)

In [20]:
def convert_examples_to_features(example_batch):
    # Convert batch inputs to list of strings explicitly
    dialogues = [str(text) for text in example_batch['dialogue']]
    summaries = [str(text) for text in example_batch['summary']]

    # Tokenize dialogue inputs
    input_encodings = tokenizer(
        dialogues,
        max_length=1024,
        truncation=True,
        padding="max_length"
    )

    # Tokenize summary targets
    target_encodings = tokenizer(
        summaries,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [21]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [22]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [23]:
dataset_samsum_pt["train"]["input_ids"][1]

[18038,
 151,
 2632,
 127,
 119,
 6228,
 118,
 115,
 136,
 2974,
 152,
 10463,
 151,
 35884,
 130,
 329,
 107,
 18038,
 151,
 2587,
 314,
 1242,
 10463,
 151,
 1509,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0

In [24]:
dataset_samsum_pt["train"]["attention_mask"][1]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [25]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [26]:
!pip install --upgrade transformers


In [27]:
import transformers
print(transformers.__version__)


4.55.3


In [28]:
from transformers import TrainingArguments

trainer_args = TrainingArguments(
    output_dir="pegasus-samsum",
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    # <- fixed key
    eval_steps=500,
    save_steps=1000000,           # <- int, not 1e6
    gradient_accumulation_steps=16,
)



## TrainingArguments in HuggingFace

`TrainingArguments` HuggingFace ke **Trainer API** ko guide karta hai ki model ko kaise train karna hai.

---
🔹 Explanation

output_dir='pegasus-samsum'

Model checkpoints, logs aur configs yahan save honge.

Training ke baad ek folder pegasus-samsum/ banega.

num_train_epochs=1

Dataset ko kitni baar train karna hai.

1 = ek round (epoch). Usually 3–5 epochs use hote hain.

warmup_steps=500

Pehle 500 steps me learning rate 0 → base LR tak slowly increase hoga.

Stable training ke liye.

per_device_train_batch_size=1

Ek GPU/CPU pe training batch size = 1.

Har step pe sirf 1 dialogue-summary pair.

per_device_eval_batch_size=1

Evaluation batch size = 1.

Memory bachane ke liye.

weight_decay=0.01

Regularization technique.

Weights thoda shrink hote hain → overfitting kam hota hai.

logging_steps=10

Har 10 steps pe training logs (loss, lr etc) print hoga.

evaluation_strategy='steps'

Evaluation har kuch steps ke baad chalegi (not just at epoch end).

eval_steps=500

Har 500 steps ke baad validation dataset pe evaluation chalega.

save_steps=1e6

Checkpoints save karne ka interval.

1,000,000 steps = practically sirf end me save karega.

gradient_accumulation_steps=16

Effective batch size bada karne ka trick.

Effective batch size = 1 × 16 = 16.

Model 16 steps tak gradients accumulate karke fir ek update karega.

🔹 Analogy

Socho tum gym me ho aur ek set = 16 pushups karne hain.

Tum 1 pushup karte ho (batch=1) → accumulate karte ho.

16 hone ke baad trainer bolta hai: “ok, ek rep complete hua.”

Yehi kaam gradient accumulation karta hai.

✅ In short:
Ye config training ka roadmap hai →

Kahaan save karna hai

Kitna train karna hai

Kab evaluate karna hai

Kaise batches handle karni hain


In [29]:
from transformers import Trainer
trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_pt["test"],
    eval_dataset=dataset_samsum_pt["validation"]
)


/tmp/ipython-input-3239901260.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🔹 Trainer ka role

HuggingFace ka Trainer ek high-level training loop hai jo deep learning model train karna simple bana deta hai.
Normally tumhe khud likhna padta hai:

forward pass

loss calculation

backpropagation

optimizer step

scheduler step

checkpoint save/load

evaluation

🔹 Har parameter ka role

model=model_pegasus

Ye wo seq2seq model hai (Pegasus) jo dialogues → summary banane ke liye train ho raha hai.

Trainer is model pe forward + backward pass karega.

args=trainer_args

Ye TrainingArguments hai jo tumne pehle banaya tha.

Isme sab cheezein defined hain:

kitne epochs

batch size

logging/eval frequency

gradient accumulation

output dir etc.

tokenizer=tokenizer

Trainer ko pata hona chahiye ki raw text ko tokens me kaise convert kare aur predictions ko wapas text me kaise decode kare.

Ye especially evaluation/metrics me helpful hai.

data_collator=seq2seq_data_collator

Batch banaate waqt har sequence ki length alag hoti hai.

DataCollatorForSeq2Seq:

input ko pad karta hai (sab equal length)

decoder labels ke padded tokens ko -100 banata hai (taaki loss calculation me ignore ho jaye)

train_dataset=dataset_samsum_pt["test"]

Ye dataset hai jis par model train karega.

Usually "train" split use hota hai, lekin tumne "test" diya hai (shayd demo ya memory constraints ki wajah se).

eval_dataset=dataset_samsum_pt["validation"]

Validation dataset jis par har evaluation step pe model ka performance check hota hai.

Ye training ke dauraan overfitting track karne ke liye hota hai.

🔹 Summary in simple analogy

Socho tum gym me ek trainer hire karte ho.

Tum (user) bolte ho: "Mujhe Pegasus ko train karna hai, ye mera plan (TrainingArguments) hai, ye mera diet (dataset) hai."

Trainer (HuggingFace Trainer) automatically:

reps (epochs) count karta hai,

har set ke baad progress note karta hai (logging),

har kuch time baad tumhara stamina check karta hai (eval on validation set),

aur jab tum goal pe pahunch jaate ho, final report (saved model) deta hai.

👉 Matlab: Trainer tumhare training loop ka automation hai.

In [30]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kushagracse-techai (kushagracse-techai-ellysiumai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,10.583100
20,10.642800
30,10.530200
40,10.487400
50,10.327600


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=52, training_loss=10.500742582174448, metrics={'train_runtime': 897.2222, 'train_samples_per_second': 0.913, 'train_steps_per_second': 0.058, 'total_flos': 2366471355236352.0, 'train_loss': 10.500742582174448, 'epoch': 1.0})

🔹 1. train/loss (Top Left)

Yeh graph batata hai ki training loss step by step kaise change ho raha hai.

Start me loss ~10.6 tha → thoda increase hua → phir decrease hone laga (achha sign hai).

Loss ka decrease matlab model input dialogues ko summary me map karna sikh raha hai.

🔹 2. train/learning_rate (Top Middle)

Yeh line warmup steps ki wajah se upar ja rahi hai.

Matlab suru me learning rate chhoti thi (0.000001), dheere-dheere barh rahi hai (by step 50 ~0.0000045).

Ye intentional hota hai taaki model stable rahe aur suru me “shock” na lage.

🔹 3. train/grad_norm (Top Right)

Gradient Norm ka matlab hai updates kitne bade hain.

Agar bahut zyada hote (spikes high hote), to training unstable ho jaati.

Tumhare case me step 20 pe spike aya (~60), phir neeche aa gaya ~15–20 ke range me → Matlab gradients settle ho gaye.

🔹 4. train/global_step (Bottom Left)

Yeh simply count kar raha hai ki kitne steps complete ho gaye.

0 → 52 tak linear increase.

Har batch ek step hota hai.

🔹 5. train/epoch (Bottom Right)

Yeh dikhata hai abhi tum kis epoch me ho.

Start → 0.2 … 0.5 … 1.0 → Matlab 1 full epoch complete ho gaya.

Tumne num_train_epochs=1 set kiya tha, isliye ye 1 tak gaya aur training ruk gayi.

🧠 Summary in Simple Terms

Loss graph: Neeche ja raha → model improve kar raha hai.

Learning rate: Warmup ke chalte steadily increase ho raha hai.

Grad norm: Pehle thoda unstable, phir settle → normal behavior.

Global step & Epoch: Training progress track karte hain.

In [31]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


Ye function banaya gaya hai trained summarization model ko test dataset pe evaluate karne ke liye using ROUGE scores.

🔹 Code Breakdown
1. generate_batch_sized_chunks
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


👉 Ye ek helper function hai jo bada dataset ko chhote chhote batches me todta hai.

Agar tumhare dataset me 1000 dialogues hain aur batch_size=16 hai → to ye function 1000 ko ~63 groups me tod dega (16 items har batch me).

yield use karne se ye ek generator ban jaata hai jo batches ek-ek karke return karega.

2. calculate_metric_on_test_ds Function
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):


Ye main function hai jo:

Dataset lega (test set).

Metric lega (jaise ROUGE).

Model aur tokenizer lega (Pegasus + tokenizer).

Fir step by step predictions karega aur compare karega gold summaries se.

3. Dataset ko batch me todna
article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))


dataset[column_text] → input dialogues (yahan pe column ka naam "article" diya hai, tumhare case me "dialogue" hoga).

dataset[column_summary] → gold summaries (tumhare case me "summary" hoga).

In dono ko generate_batch_sized_chunks se tod diya batch_size=16 ke according.

4. Loop on batches
for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):


Har batch ke liye ek saath article_batch (16 dialogues) aur target_batch (16 gold summaries) nikalega.

tqdm bas ek progress bar show karta hai loop ke liye.

5. Tokenize inputs
inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                   padding="max_length", return_tensors="pt")


Har batch of dialogues ko tokens me convert kiya (model ko samajhne ke liye).

Max input length 1024 rakhi hai.

return_tensors="pt" → PyTorch tensor bana ke return karega.

6. Generate model summaries
summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                           attention_mask=inputs["attention_mask"].to(device),
                           length_penalty=0.8, num_beams=8, max_length=128)


model.generate() → Pegasus decoder se summary banayi ja rahi hai.

Parameters:

length_penalty=0.8 → chhoti concise summary promote karega (long sentences ko discourage karega).

num_beams=8 → Beam search use karega (zyada accurate summary ke liye multiple candidate paths explore karega).

max_length=128 → summary ki max length.

7. Decode token IDs → text
decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                      clean_up_tokenization_spaces=True)
                     for s in summaries]
decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


Model ne summary ko IDs (integers) me diya hota hai.

tokenizer.decode() unhe wapas human-readable text me convert karta hai.

skip_special_tokens=True → [PAD], [EOS] jaise tokens hata dega.

8. Metric calculation
metric.add_batch(predictions=decoded_summaries, references=target_batch)


Predictions (model ki summaries) aur References (gold summaries) ko metric me daal diya.

Metric = ROUGE (Recall-Oriented Understudy for Gisting Evaluation).

9. Final score
score = metric.compute()
return score


Pure dataset process karne ke baad metric compute hoga.

Output me tumhe ROUGE-1, ROUGE-2, ROUGE-L scores milenge.

Ye batayega model ki summaries kitni close hain gold summaries se.

🔹 In Short (Flow)

Dataset ko chhote batches me divide karo.

Har batch → tokenize → model se summary generate → decode.

Gold summaries se compare karo.

Metric compute karke ROUGE score nikalo.

👉 Matlab ye function test dataset par Pegasus model ka summarization quality measure karta hai.

In [33]:
from evaluate import load

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load("rouge")


In [35]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)
rouge_dict = {rn: score[rn] for rn in rouge_names}
pd.DataFrame(rouge_dict, index=["pegasus"])


100%|██████████| 5/5 [00:30<00:00,  6.01s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.019032,0.0,0.018904,0.019013


🔹 ROUGE Kya Batata Hai?

ROUGE batata hai ki tumhare model ka generated summary kitna close hai human-written (gold/reference) summary ke.

Wo overlap check karta hai between:

Predicted summary (model output)

Gold summary (human reference)

Overlap = words, phrases, or sequences jo dono me common hain.

🔹 Types of ROUGE

ROUGE-1

Unigram overlap (single words).

Example:

Gold: "The cat is on the mat"

Pred: "The cat sat on mat"

Overlap words: "The, cat, on, mat" → Score.

ROUGE-2

Bigram overlap (two consecutive words).

Example: "the cat", "cat is" …

ROUGE-L

Longest Common Subsequence (LCS).

Dekhta hai dono summaries me longest sequence kitna same hai.

ROUGE-Lsum

ROUGE-L ka summarization-specific version (sentence-level).

🔹 Metrics Inside ROUGE

ROUGE ke andar 3 cheezein hoti hain:

Precision = of the words I generated, kitne gold me bhi the?

Recall = of the words in the gold summary, kitne model ne generate kiye?

F1 (fmeasure) = balance of precision + recall.

🔹 Step by Step Explanation

calculate_metric_on_test_ds(...)

Tum apna evaluation function chala rahe ho.

Ye model ko test dataset ke 10 dialogues (0 se 9 tak) pe summaries generate karne ke liye bolta hai.

Fir un summaries ko ROUGE metric se compare karta hai gold (human) summaries ke saath.

Output → ek score object jisme ROUGE-1, ROUGE-2, ROUGE-L, ROUGE-Lsum ke values hain.

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

Ye loop ek dictionary banata hai:

{
  "rouge1": <score value>,
  "rouge2": <score value>,
  "rougeL": <score value>,
  "rougeLsum": <score value>
}


.mid.fmeasure → Matlab ROUGE ke result me precision, recall, f-measure teen hote hain.

Tum sirf F1-score (fmeasure) le rahe ho (jo balance karta hai precision + recall).

pd.DataFrame(rouge_dict, index=['pegasus'])

Ye dictionary ko ek table format (DataFrame) me convert kar raha hai.

Row ka naam diya "pegasus" (kyunki tum Pegasus model test kar rahe ho).

In [36]:
## Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [37]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [38]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [39]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .


In [41]:
paragraph = """The rapid growth of social media platforms has transformed the way people communicate, share information, and interact with the world. Every day, billions of posts, comments, and messages are generated across networks like Twitter, Facebook, and Instagram. While this has created new opportunities for global connectivity, it has also led to challenges such as misinformation, online harassment, and information overload. For businesses, social media has become both a tool and a challenge: on one hand, it provides direct access to customers and real-time feedback, while on the other hand, it requires constant monitoring and content moderation. Researchers and developers are now focusing on advanced Natural Language Processing (NLP) techniques to analyze, summarize, and filter this massive stream of unstructured text data. By automatically generating summaries, companies and individuals can quickly extract insights without spending hours reading through thousands of posts. This not only saves time but also enhances decision-making in fast-moving digital environments."""


result = pipe(paragraph, **gen_kwargs)[0]["summary_text"]
result = result.replace("<n>", " ")

print("Paragraph:\n", paragraph)
print("\nSummary:\n", result)


Paragraph:
 The rapid growth of social media platforms has transformed the way people communicate, share information, and interact with the world. Every day, billions of posts, comments, and messages are generated across networks like Twitter, Facebook, and Instagram. While this has created new opportunities for global connectivity, it has also led to challenges such as misinformation, online harassment, and information overload. For businesses, social media has become both a tool and a challenge: on one hand, it provides direct access to customers and real-time feedback, while on the other hand, it requires constant monitoring and content moderation. Researchers and developers are now focusing on advanced Natural Language Processing (NLP) techniques to analyze, summarize, and filter this massive stream of unstructured text data. By automatically generating summaries, companies and individuals can quickly extract insights without spending hours reading through thousands of posts. This 